# Mortality Analysis
Analyze data from the National Notifiable Diseases Surveillance System on [deaths by pneumonia and influenza](https://www.healthdata.gov/dataset/nndss-table-ii-invasive-pneumococcal-diseases-all-ages) in 122 cities.

In [44]:
import numpy as np
from scipy import stats
import cntk as C

In [45]:
dataPath = "source.csv"
data = np.genfromtxt(dataPath, delimiter = ",", dtype = str)

In [46]:
deaths = data[1:, 6]
print(deaths)

non_blanks = []
for i in range(len(deaths)):
    if deaths[i] != "":
        non_blanks.append(i)
        
deaths = deaths[non_blanks]
years = stats.zscore( (data[1:, 0])[non_blanks].astype(np.float32))
winter = np.asarray( [1.0 if (week < 7 or 47 < week) else 0.0 for week in data[1:, 1].astype(int)], dtype = np.float32 )
winter = winter[non_blanks]

['11' '11' '5' ..., '2' '12' '11']


In [47]:
def shuffle_arrays(*arrays):
    rand_state = np.random.get_state()
    for array in arrays:
        np.random.set_state(rand_state)
        np.random.shuffle(array)

shuffle_arrays(years, winter, deaths)

In [48]:
num_samples = len(years)
training_size = num_samples * 2 // 3;


In [49]:
num_features = 2
num_classes = 1 #Do I even need this, since this is a regression problem?
input_var = C.input_variable(num_features, np.float32)

In [50]:
parm_dict = {}
def linear_layer(input_var, num_output_classes):
    
    n = input_var.shape[0]
    
    weights = C.parameter(shape = (input_var.shape[0], num_output_classes))
    biases = C.parameter(shape = (num_output_classes))
    
    return C.times(input_var, weights) + biases

In [51]:
z = linear_layer(input_var, num_classes)
label = C.input_variable(num_classes, np.float32)
loss = C.squared_error(z, label)

In [56]:
learning_rate = 0.5
minibatch_size = 25

lr_schedule = C.learning_rate_schedule(learning_rate, C.UnitType.minibatch)
learner = C.sgd(z.parameters, lr_schedule)

trainer = C.trainer(z, (loss), [learner])

TypeError: 'module' object is not callable

In [57]:
print(z)
print(type(z))
print(loss)
print(type(loss))
print(learner)
print(type(learner))

Composite(Tensor[2]) -> Tensor[1]
<class 'cntk.ops.functions.Function'>
Composite(Tensor[2], Tensor[1]) -> np.float32
<class 'cntk.ops.functions.Function'>
<cntk.learners.Learner; proxy of <Swig Object of type 'CNTK::LearnerPtr *' at 0x000001ADE5A04DB0> >
<class 'cntk.learners.Learner'>
